# Advent of Code 2024

In [ ]:
from aocd.models import Puzzle
from pathlib import Path
puzzle = Puzzle(year=2024, day=int(Path(__vsc_ipynb_file__).stem))
puzzle.url

# Part 1

In [3]:
example = """########
#..O.O.#
##@.O..#
#...O..#
#.#.O..#
#...O..#
#......#
########

<^^>>>vv<v>>v<<"""

In [36]:
def parse_input(input_data):
    g, m = input_data.split('\n\n')
    g = g.splitlines()
    w = len(g[0])
    h = len(g)
    grid = bytearray(w * h)
    for r, line in enumerate(g):
        for c, char in enumerate(line):
            grid[r * w + c] = ord(char)
    moves = m.replace('\n', '')
    return grid, w, h, moves

def display(grid, w, h, rx, ry):
    temp = grid[ry * w + rx]
    assert(temp == ord('.'))
    grid[ry * w + rx] = ord('@')
    for y in range(h):
        print(grid[y*w:(y+1)*w].decode())
    grid[ry * w + rx] = temp


def solve_a(input_data):
    grid, w, h, moves = parse_input(input_data)
    rx, ry = 0, 0
    for y in range(h):
        for x in range(w):
            if grid[y * w + x] == ord('@'):
                rx, ry = x, y
                grid[y * w + x] = ord('.')
                break
    d = {
        '^': (0, -1),
        'v': (0, 1),
        '<': (-1, 0),
        '>': (1, 0),
    }
    for i,m in enumerate(moves):
        # print(i, m)
        # display(grid, w, h, rx, ry)
        dx, dy = d[m]
        nx, ny = rx + dx, ry + dy
        if 0 <= nx < w and 0 <= ny < h and grid[ny * w + nx] != ord('#'):
            num_boxes = 0
            bx, by = nx, ny
            while grid[by * w + bx] == ord('O'):
                num_boxes += 1
                bx, by = bx + dx, by + dy
            if grid[by * w + bx] == ord('.'):
                grid[ny * w + nx] = ord('.')
                if num_boxes > 0:
                    grid[by * w + bx] = ord('O')
                rx, ry = nx, ny
    
    # display(grid, w, h, rx, ry)

    total = 0
    for y in range(h):
        for x in range(w):
            if grid[y * w + x] == ord('O'):
                total += y * 100 + x
    return total

puzzle.answer_a = solve_a(puzzle.input_data)


# Part 2

In [30]:
small_example = """#######
#...#.#
#.....#
#..OO@#
#..O..#
#.....#
#######

<vv<<^^<<^^"""

In [ ]:
def solve_b(input_data):
    grid, w, h, moves = parse_input(input_data)
    nw = w * 2
    nh = h
    ngrid = bytearray(nw * nh)
    for y in range(h):
        for x in range(w):
            val = grid[y * w + x]
            if val == ord('#'):
                ngrid[y * nw + x * 2] = ord('#')
                ngrid[y * nw + x * 2 + 1] = ord('#')
            elif val == ord('O'):
                ngrid[y * nw + x * 2] = ord('[')
                ngrid[y * nw + x * 2 + 1] = ord(']')
            elif val == ord('.'):
                ngrid[y * nw + x * 2] = ord('.')
                ngrid[y * nw + x * 2 + 1] = ord('.')
            elif val == ord('@'):
                ngrid[y * nw + x * 2] = ord('@')
                ngrid[y * nw + x * 2 + 1] = ord('.')
    
    rx, ry = 0, 0
    for y in range(nh):
        for x in range(nw):
            if ngrid[y * nw + x] == ord('@'):
                rx, ry = x, y
                ngrid[y * nw + x] = ord('.')
                break
    
    d = {
        '^': (0, -1),
        'v': (0, 1),
        '<': (-1, 0),
        '>': (1, 0),
    }
    def get(x,y):
        return ngrid[y * nw + x]
    def put(x,y,c):
        ngrid[y * nw + x] = c
    def move_from_to(x,y,x2,y2):
        assert(get(x2,y2) == ord('.'))
        put(x2,y2,get(x,y))
        put(x,y, ord('.'))
    def is_box(bx, by):
        c = get(bx,by)
        return c == ord('[') or c == ord(']')
    def expand_box_row(box_row, by, dy):
        """For a given row of boxes, and set of x positions that have a pushed box, at row by, see what it pushes on the next row in direction dy. Return new comb, or ."""
        nby = by + dy
        if nby < 0 or nby >= nh:
            return None
        new_box_row = set()
        # May need to expand bx0 and bx1
        for bx in box_row:
            c = get(bx, nby)
            if c == ord('#'):
                return None
            elif c == ord('['):
                new_box_row.add(bx)
                new_box_row.add(bx+1)
            elif c == ord(']'):
                new_box_row.add(bx-1)
                new_box_row.add(bx)
        return new_box_row
    
    print('Initial state:')
    display(ngrid, nw, h, rx, ry)

    for i, m in enumerate(moves):
        dx, dy = d[m]
        nx, ny = rx + dx, ry + dy
        if 0 <= nx < nw and 0 <= ny < nh and get(nx,ny) != ord('#'):
            if dx != 0:
                bx, by = nx, ny
                # Horizontal move
                box_count = 0
                while is_box(bx, by):
                    box_count += 1
                    bx, by = bx + dx, by + dy
                if get(bx,by) == ord('.'):
                    for i in range(box_count):
                        move_from_to(bx-(i+1)*dx, by, bx-i*dx, by)
                    rx, ry = nx, ny
            else:
                # Vertical move
                def calc_box_moves():
                    by = ny
                    box_moves = [(ry, set([rx]))]
                    while True:
                        y, boxes = box_moves[-1]
                        new_box_row = expand_box_row(boxes, y, dy)
                        if new_box_row == None:
                            return None
                        if len(new_box_row) == 0:
                            return box_moves
                        box_moves.append((y + dy, new_box_row))
                box_moves = calc_box_moves()
                if box_moves != None:
                    for sy, boxes in reversed(box_moves):
                        for bx in boxes:
                            move_from_to(bx, sy, bx, sy+dy)
                    rx, ry = nx, ny
        print('Move ', i, m)
        display(ngrid, nw, h, rx, ry)

    display(ngrid, nw, h, rx, ry)

    
    total = 0
    for y in range(nh):
        for x in range(nw):
            if ngrid[y * nw + x] == ord('['):
                total += y * 100 + x
    return total

solve_b(small_example)

In [41]:
larger_example="""##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########

<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^
vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v
><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<
<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^
^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><
^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^
>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^
<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>
^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>
v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^"""


In [ ]:
solve_b(larger_example)

In [ ]:
puzzle.answer_b = solve_b(puzzle.input_data)